In [80]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import mean_squared_error

# Description

A continuación, voy a aplicar el modelo RandomForest, SIN escalar. 
Sin embargo, estas serán las features 
- Calcular el tamaño como la multiplicación de las columnas x, y, z.
- Eliminamos posteriormente estas columnas x, y, z.
- Agrupación de la columna de colores donde J es el peor y D es el mejor. 
- La columna de cut haremos un label enconding donde 0 sea Fair y 4 sea Ideal. 
- Mantengo la clasificación de city por cotinent. 
- Eliminamos la columna de depth.
- Mantengo la clasificación de clarity en 4 columnas. 




In [26]:
diamonds = pd.read_csv('./data/diamonds.csv')

In [27]:
diamonds.head()

,price,carat,city,depth,table,x,y,z,cut,color,clarity
0,4268,1.21,Dubai,62.4,58.0,6.83,6.79,4.25,Premium,J,VS2
1,505,0.32,Kimberly,63.0,57.0,4.35,4.38,2.75,Very Good,H,VS2
2,2686,0.71,Las Vegas,65.5,55.0,5.62,5.53,3.65,Fair,G,VS1
3,738,0.41,Kimberly,63.8,56.0,4.68,4.72,3.00,Good,D,SI1
4,4882,1.02,Dubai,60.5,59.0,6.55,6.51,3.95,Ideal,G,SI1


In [28]:
diamonds["size"] = diamonds["x"] * diamonds["y"] * diamonds["z"]

In [29]:
diamonds = diamonds.drop('x', axis=1).drop('y', axis=1).drop('z', axis=1)

In [30]:
diamonds['cut'] = diamonds['cut'].map({'Fair': 1, 
                                       'Good': 2, 
                                       'Very Good': 3, 
                                       'Premium': 4, 
                                       'Ideal': 5})

In [31]:
diamonds['color'] = diamonds['color'].map({'J': 1,
                                          'I': 2,
                                          'H': 3, 
                                          'G': 4,
                                          'F': 5, 
                                          'E': 6,
                                          'D': 7})


In [32]:
diamonds['continent'] = diamonds['city'].map({'Dubai': 'Asia',
                                                'Kimberly': 'Africa',
                                                'Las Vegas': 'America',
                                                'Tel Aviv': 'Asia',
                                                'Amsterdam': 'Europe',
                                                'Zurich': 'Europe',
                                                'Antwerp': 'Europe',
                                                'Madrid': 'Europe',
                                                'Paris': 'Europe',
                                                'Surat': 'Asia',
                                                'Luxembourg': 'Europe',
                                                'London': 'Europe',
                                                'New York City': 'America'})

In [33]:
diamonds = diamonds.drop('city', axis=1)

In [34]:
diamonds['clarity'] = diamonds['clarity'].replace(['IF'], ['VVSI']).replace(['VVS1'], ['VVSI']).replace(['VVS2'], ['VVSI'])
# VVSI = Very Very Sligthly Included, incluye IF (Internally Flawless, VVS1 y VVS2 (Very, Very Slightly Included 1 y 2))

In [35]:
diamonds['clarity'] = diamonds['clarity'].replace(['VS1'], ['VSI']).replace(['VS2'], ['VSI'])
# VVS = Very Slightly Included, incluye Very Slightly Included 1 y 2 

In [36]:
diamonds['clarity'] = diamonds['clarity'].replace(['SI1'], ['SI']).replace(['SI2'], ['SI'])
# SI = Slightly Included, incluye Slightly Included 1 y 2 

In [37]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      40455 non-null  int64  
 1   carat      40455 non-null  float64
 2   depth      40455 non-null  float64
 3   table      40455 non-null  float64
 4   cut        40455 non-null  int64  
 5   color      40455 non-null  int64  
 6   clarity    40455 non-null  object 
 7   size       40455 non-null  float64
 8   continent  40455 non-null  object 
dtypes: float64(4), int64(3), object(2)
memory usage: 2.8+ MB


## One Hot Encoding

In [38]:
diamonds_dummy = pd.get_dummies(diamonds, drop_first=True, dtype=float)

In [39]:
diamonds_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   price              40455 non-null  int64  
 1   carat              40455 non-null  float64
 2   depth              40455 non-null  float64
 3   table              40455 non-null  float64
 4   cut                40455 non-null  int64  
 5   color              40455 non-null  int64  
 6   size               40455 non-null  float64
 7   clarity_SI         40455 non-null  float64
 8   clarity_VSI        40455 non-null  float64
 9   clarity_VVSI       40455 non-null  float64
 10  continent_America  40455 non-null  float64
 11  continent_Asia     40455 non-null  float64
 12  continent_Europe   40455 non-null  float64
dtypes: float64(10), int64(3)
memory usage: 4.0 MB


In [40]:
diamonds_dummy.head()

,price,carat,depth,table,cut,color,size,clarity_SI,clarity_VSI,clarity_VVSI,continent_America,continent_Asia,continent_Europe
0,4268,1.21,62.4,58.0,4,1,197.096725,0.0,1.0,0.0,0.0,1.0,0.0
1,505,0.32,63.0,57.0,3,3,52.395750,0.0,1.0,0.0,0.0,0.0,0.0
2,2686,0.71,65.5,55.0,1,4,113.436890,0.0,1.0,0.0,1.0,0.0,0.0
3,738,0.41,63.8,56.0,2,7,66.268800,1.0,0.0,0.0,0.0,0.0,0.0
4,4882,1.02,60.5,59.0,5,4,168.429975,1.0,0.0,0.0,0.0,1.0,0.0


## ⚠️⚠️ DECISION = SIN ESCALAR

In [41]:
X = diamonds_dummy.drop('price', axis=1)
y = diamonds_dummy['price']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
# model = GradientBoostingRegressor(random_state = 42)   # No necesita one hot encoding
# model = AdaBoostRegressor(random_state = 42)
# model = BaggingRegressor(random_state = 42)

In [59]:
model.fit(X_train, y_train)

TypeError: StackingRegressor.fit() missing 1 required positional argument: 'y'

In [56]:
# predict_GB = model.predict(X_test)
# predict_AB = model.predict(X_test)
predict_BR = model.predict(X_test)

In [57]:
# RMSE calculation

# rmse_GB = mean_squared_error(y_test, predict_GB, squared=False)
# rmse_AB = mean_squared_error(y_test, predict_AB, squared=False)
rmse_BR = mean_squared_error(y_test, predict_BR, squared=False)
print(rmse_GB)
print(rmse_AB)
print(rmse_BR)

775.7627338940251
1186.9217004101906
697.6000872411721


C:\Users\tete1\miniconda3\envs\jupyter_env\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## ⚠️⚠️ DECISION = SOLO LABEL ENCODING

In [57]:
diamonds2 = pd.read_csv('./data/diamonds.csv')

In [58]:
diamonds2["size"] = diamonds2["x"] * diamonds2["y"] * diamonds2["z"]

In [59]:
diamonds2 = diamonds2.drop('x', axis=1).drop('y', axis=1).drop('z', axis=1)

In [60]:
diamonds2['cut'] = diamonds2['cut'].map({'Fair': 1, 
                                       'Good': 2, 
                                       'Very Good': 3, 
                                       'Premium': 4, 
                                       'Ideal': 5})

In [61]:
diamonds2['color'] = diamonds2['color'].map({'J': 1,
                                          'I': 2,
                                          'H': 3, 
                                          'G': 4,
                                          'F': 5, 
                                          'E': 6,
                                          'D': 7})

In [62]:
diamonds2['continent'] = diamonds2['city'].map({'Dubai': 'Asia',
                                                'Kimberly': 'Africa',
                                                'Las Vegas': 'America',
                                                'Tel Aviv': 'Asia',
                                                'Amsterdam': 'Europe',
                                                'Zurich': 'Europe',
                                                'Antwerp': 'Europe',
                                                'Madrid': 'Europe',
                                                'Paris': 'Europe',
                                                'Surat': 'Asia',
                                                'Luxembourg': 'Europe',
                                                'London': 'Europe',
                                                'New York City': 'America'})

In [63]:
diamonds2 = diamonds2.drop('city', axis=1)

In [64]:
diamonds2['continent'].value_counts()

continent
Europe     20168
Asia       10139
America     5172
Africa      4976
Name: count, dtype: int64

In [65]:
diamonds2['continent'] = diamonds2['continent'].map({'Africa': 1,
                                          'Europe': 2,
                                          'America': 3, 
                                          'Asia': 4})

In [66]:
diamonds2['clarity'].value_counts()

clarity
SI1     9749
VS2     9248
SI2     6929
VS1     6066
VVS2    3799
VVS1    2774
IF      1327
I1       563
Name: count, dtype: int64

In [67]:
diamonds2['clarity'] = diamonds2['clarity'].map({'I1': 1,
                                          'SI1': 2,
                                          'SI2': 3, 
                                          'VS1': 4,
                                          'VS2': 5, 
                                          'VVS1': 6,
                                          'VVS2': 7,
                                                'IF': 8})

In [68]:
diamonds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      40455 non-null  int64  
 1   carat      40455 non-null  float64
 2   depth      40455 non-null  float64
 3   table      40455 non-null  float64
 4   cut        40455 non-null  int64  
 5   color      40455 non-null  int64  
 6   clarity    40455 non-null  int64  
 7   size       40455 non-null  float64
 8   continent  40455 non-null  int64  
dtypes: float64(4), int64(5)
memory usage: 2.8 MB


In [69]:
X = diamonds2.drop('price', axis=1)
y = diamonds2['price']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
# model = GradientBoostingRegressor(random_state = 42)   # No necesita one hot encoding
# model = AdaBoostRegressor(random_state = 42)
model = BaggingRegressor(random_state = 42)

In [82]:
model.fit(X_train, y_train)

BaggingRegressor(random_state=42)

In [83]:
# predict_GB_Label = model.predict(X_test)
# predict_AB_label = model.predict(X_test)
predict_BR_label = model.predict(X_test)

In [84]:
# RMSE calculation

# rmse_GB_label = mean_squared_error(y_test, predict_GB_Label, squared=False)
# rmse_AB_label = mean_squared_error(y_test, predict_AB_label, squared=False)
rmse_BR_label = mean_squared_error(y_test, predict_BR_label, squared=False)
print(rmse_GB_label)
print(rmse_AB_label)
print(rmse_BR_label)

670.2618950777608
1116.279244584736
564.5579118520077


C:\Users\tete1\miniconda3\envs\jupyter_env\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Aplico a diamonds_test el último

In [94]:
diamonds_test = pd.read_csv('./data/diamonds_test.csv')

In [95]:
diamonds_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       13485 non-null  int64  
 1   carat    13485 non-null  float64
 2   cut      13485 non-null  object 
 3   color    13485 non-null  object 
 4   clarity  13485 non-null  object 
 5   depth    13485 non-null  float64
 6   table    13485 non-null  float64
 7   x        13485 non-null  float64
 8   y        13485 non-null  float64
 9   z        13485 non-null  float64
 10  city     13485 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.1+ MB


In [97]:
diamonds_test = diamonds_test.drop('id', axis=1)

In [98]:
diamonds_test["size"] = diamonds_test["x"] * diamonds_test["y"] * diamonds_test["z"]

In [99]:
diamonds_test = diamonds_test.drop('x', axis=1).drop('y', axis=1).drop('z', axis=1)

In [100]:
diamonds_test['cut'] = diamonds_test['cut'].map({'Fair': 1, 
                                       'Good': 2, 
                                       'Very Good': 3, 
                                       'Premium': 4, 
                                       'Ideal': 5})

In [101]:
diamonds_test['color'] = diamonds_test['color'].map({'J': 1,
                                          'I': 2,
                                          'H': 3, 
                                          'G': 4,
                                          'F': 5, 
                                          'E': 6,
                                          'D': 7})

In [102]:
diamonds_test['continent'] = diamonds_test['city'].map({'Dubai': 'Asia',
                                                'Kimberly': 'Africa',
                                                'Las Vegas': 'America',
                                                'Tel Aviv': 'Asia',
                                                'Amsterdam': 'Europe',
                                                'Zurich': 'Europe',
                                                'Antwerp': 'Europe',
                                                'Madrid': 'Europe',
                                                'Paris': 'Europe',
                                                'Surat': 'Asia',
                                                'Luxembourg': 'Europe',
                                                'London': 'Europe',
                                                'New York City': 'America'})

In [104]:
diamonds_test = diamonds_test.drop('city', axis=1)

In [105]:
diamonds_test['continent'] = diamonds_test['continent'].map({'Africa': 1,
                                          'Europe': 2,
                                          'America': 3, 
                                          'Asia': 4})

In [106]:
diamonds_test['clarity'] = diamonds_test['clarity'].map({'I1': 1,
                                          'SI1': 2,
                                          'SI2': 3, 
                                          'VS1': 4,
                                          'VS2': 5, 
                                          'VVS1': 6,
                                          'VVS2': 7,
                                                'IF': 8})

In [107]:
diamonds_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   carat      13485 non-null  float64
 1   cut        13485 non-null  int64  
 2   color      13485 non-null  int64  
 3   clarity    13485 non-null  int64  
 4   depth      13485 non-null  float64
 5   table      13485 non-null  float64
 6   size       13485 non-null  float64
 7   continent  13485 non-null  int64  
dtypes: float64(4), int64(4)
memory usage: 842.9 KB


In [110]:
diamonds_test.head()

,carat,cut,color,clarity,depth,table,size,continent
0,0.79,3,5,2,62.7,60.0,125.806866,2
1,1.20,5,1,4,61.0,57.0,196.129362,4
2,1.57,4,3,2,62.2,61.0,246.878712,1
3,0.90,3,5,2,63.8,54.0,145.593630,1
4,0.50,3,5,4,62.9,58.0,81.997355,2


In [111]:
diamonds_test = diamonds_test[['carat', 'depth', 'table', 'cut', 'color', 'clarity', 'size', 'continent']]

In [112]:
diamonds_test.head()

,carat,depth,table,cut,color,clarity,size,continent
0,0.79,62.7,60.0,3,5,2,125.806866,2
1,1.20,61.0,57.0,5,1,4,196.129362,4
2,1.57,62.2,61.0,4,3,2,246.878712,1
3,0.90,63.8,54.0,3,5,2,145.593630,1
4,0.50,62.9,58.0,3,5,4,81.997355,2


In [113]:
predict_BR_label_sub = model.predict(diamonds_test)

In [114]:
predict_BR_label_sub = pd.DataFrame(predict_BR_label_sub, columns=['price'])

In [115]:
predict_BR_label_sub['id'] = range(0, len(predict_BR_label_sub))
predict_BR_label_sub = predict_BR_label_sub[['id', 'price']]
predict_BR_label_sub

,id,price
0,0,2957.8
1,1,5249.2
2,2,9257.9
3,3,4076.5
4,4,1725.4
...,...,...
13480,13480,1685.4
13481,13481,2483.5
13482,13482,3254.1
13483,13483,2010.2


In [116]:
predict_BR_label_sub.to_csv('./submissions/submission_predict_BR_label.csv', index=False)